<a href="https://colab.research.google.com/github/Kishan-Kumar-Zalavadia/AgenticAI_Experiments/blob/main/1_Quasi_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Quasi-Agent

For practice, we are going to write a quasi-agent that can write Python functions based on user requirements. It isn’t quite a real agent, it can’t react and adapt, but it can do something useful for us.

The quasi-agent will ask the user what they want code for, write the code for the function, add documentation, and finally include test cases using the unittest framework. This exercise will help you understand how to maintain context across multiple prompts and manage the information flow between the user and the LLM. It will also help you understand the pain of trying to parse and handle the output of an LLM that is not always consistent.



**Practice Exercise**

This exercise will allow you to practice programmatically sending prompts to an LLM and managing memory.

For this exercise, you should write a program that uses sequential prompts to generate any Python function based on user input. The program should:

1. First Prompt:

    - Ask the user what function they want to create
    - Ask the LLM to write a basic Python function based on the user’s description
    - Store the response for use in subsequent prompts
    - Parse the response to separate the code from the commentary by the LLM

2. Second Prompt:

    - Pass the code generated from the first prompt
    - Ask the LLM to add comprehensive documentation including:
      - Function description
      - Parameter descriptions
      - Return value description
      - Example usage
      - Edge cases

3. Third Prompt:

    - Pass the documented code generated from the second prompt
    - Ask the LLM to add test cases using Python’s unittest framework
    - Tests should cover:
      - Basic functionality
      - Edge cases
      - Error cases
      - Various input scenarios

Requirements:

  - Use the LiteLLM library
  - Maintain conversation context between prompts
  - Print each step of the development process
  - Save the final version to a Python file

If you want to practice further, try using the system message to force the LLM to always output code that has a specific style or uses particular libraries.

In [2]:
!!pip install litellm

['Collecting litellm',
 '  Downloading litellm-1.78.3-py3-none-any.whl.metadata (42 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/42.9 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b\x1b━\x1b \x1b41.0/42.9 kB\x1b \x1b130.0 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b42.9/42.9 kB\x1b \x1b917.8 kB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hRequirement already satisfied: aiohttp>=3.10 in /usr/local/lib/python3.12/dist-packages (from litellm) (3.13.0)',
 'Requirement already satisfied: click in /usr/local/lib/python3.12/dist-packages (from litellm) (8.3.0)',
 'Collecting fastuuid>=0.13.0 (from litellm)',
 '  Downloading fastuuid-0.13.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.0 kB)',
 'Requirement already satisfied: httpx>=0.23.0 in /usr/local/lib/python3.12/dist-packages (from litellm) (0.28.1)',
 'Requirement already s

In [3]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [4]:
from litellm import completion
from typing import List, Dict

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="groq/llama-3.3-70b-versatile",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


def extract_code_block(response: str) -> str:
    """Extract code block from response"""
    if '```' not in response:
        return response

    code_block = response.split('```')[1].strip()

    # Remove "python" prefix if present
    if code_block.startswith("python"):
        code_block = code_block[6:].strip()

    return code_block


def develop_custom_function():
    """Main function to interactively develop and enhance a Python function"""

    # Step 1: Get user input
    print("\nWhat kind of function would you like to create?")
    print("Example: 'A function that calculates the factorial of a number'")
    function_description = input("Your description: ").strip()

    # Initialize conversation
    messages = [
        {"role": "system", "content": "You are a Python expert helping to develop a function."}
    ]

    # Step 2: Generate initial function
    messages.append({
        "role": "user",
        "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."
    })

    initial_function = generate_response(messages)
    initial_function = extract_code_block(initial_function)

    print("\n=== Initial Function ===")
    print(initial_function)

    # Add assistant's last code-only reply to maintain context
    messages.append({"role": "assistant", "content": f"```python\n{initial_function}\n```"})

    # Step 3: Add documentation
    messages.append({
        "role": "user",
        "content": (
            "Add comprehensive documentation to this function, including description, parameters, "
            "return value, examples, and edge cases. Output the function in a ```python code block```."
        )
    })

    documented_function = generate_response(messages)
    documented_function = extract_code_block(documented_function)

    print("\n=== Documented Function ===")
    print(documented_function)

    messages.append({"role": "assistant", "content": f"```python\n{documented_function}\n```"})

    # Step 4: Add test cases
    messages.append({
        "role": "user",
        "content": (
            "Add unittest test cases for this function, including tests for basic functionality, "
            "edge cases, error cases, and various input scenarios. Output the code in a ```python code block```."
        )
    })

    test_cases = generate_response(messages)
    test_cases = extract_code_block(test_cases)

    print("\n=== Test Cases ===")
    print(test_cases)

    # Step 5: Save to file
    filename = function_description.lower()
    filename = ''.join(c for c in filename if c.isalnum() or c.isspace())
    filename = filename.replace(' ', '_')[:30] + '.py'

    with open(filename, 'w') as f:
        f.write(documented_function + '\n\n' + test_cases)

    print(f"\nFinal code has been saved to {filename}")
    return documented_function, test_cases, filename


if __name__ == "__main__":
    function_code, tests, filename = develop_custom_function()



What kind of function would you like to create?
Example: 'A function that calculates the factorial of a number'
Your description: A function to find sum of n natural numbers

=== Initial Function ===
def sum_of_natural_numbers(n):
    """
    This function calculates the sum of the first n natural numbers.
    
    Parameters:
    n (int): The number of natural numbers to sum up.
    
    Returns:
    int: The sum of the first n natural numbers.
    """
    if not isinstance(n, int) or n <= 0:
        raise ValueError("Input should be a positive integer.")
    
    # Using the formula for the sum of an arithmetic series: n * (n + 1) / 2
    return n * (n + 1) // 2

# Example usage:
print(sum_of_natural_numbers(10))  # Output: 55

=== Documented Function ===
def sum_of_natural_numbers(n):
    """
    This function calculates the sum of the first n natural numbers.

    The sum of the first n natural numbers can be calculated using the formula: 
    1 + 2 + 3 + ... + n = n * (n + 1) / 2

# Understanding the Architecture

Our quasi-agent works through three key steps:

1. Initial code generation
2. Documentation enhancement
3. Test case creation

The magic happens in how we maintain context between these steps, ensuring each builds on the previous results.

Core Components
Let’s break down the key pieces:



```
def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="groq/llama-3.3-70b-versatile",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content
```

This function handles our LLM interactions using ChatML format. Each message includes a role (“system”, “user”, or “assistant”) and content.

```
def extract_code_block(response: str) -> str:
    """Extract code block from response"""
    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()
    if code_block.startswith("python"):
        code_block = code_block[6:]

    return code_block
```

The LLM often includes commentary with its code. This function extracts just the code block, making it easier to build upon in subsequent prompts.

# The Development Process

The main function, develop_custom_function(), orchestrates three phases of development:

## Phase 1: Initial Code Generation
```
messages = [
    {"role": "system", "content": "You are a Python expert helping to develop a function."}
]

messages.append({
    "role": "user",
    "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."
})
```

We start with a system message establishing the LLM’s role, then request initial code based on the user’s description.

# Phase 2: Documentation Enhancement

```
messages.append({
    "role": "assistant",
    "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"
})

messages.append({
    "role": "user",
    "content": "Add comprehensive documentation to this function..."
})
```

Notice how we feed back the code but strip any commentary. This keeps the LLM focused on just the code structure.


# Phase 3: Test Case Generation
```
messages.append({
    "role": "assistant",
    "content": "\`\`\`python\n\n"+documented_function+"\n\n\`\`\`"
})

messages.append({
    "role": "user",
    "content": "Add unittest test cases for this function..."
})
```
Again, we maintain clean context by showing only the documented code.



# Memory Management Through Message History

The key insight is how we manage “memory” through the messages list. Each step builds on previous responses, but we carefully control what the LLM sees:

1. We only show the code, not the commentary
2. Each message provides specific instruction for the next enhancement
3. The context builds progressively through the message history
For
 example, when adding documentation, the LLM sees:

- It’s a Python expert (system message)
- The original code (previous response)
- The request for documentation (current task)
This focused context helps ensure consistent, high-quality output.


# Usage Example
Here’s how it works in practice:

```
>>> function_code, tests, filename = develop_custom_function()
What kind of function would you like to create?
Example: 'A function that calculates the factorial of a number'
Your description: Calculate fibonacci sequence up to n

=== Initial Function ===
def fibonacci(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    sequence = [0, 1]
    while len(sequence) < n:
        sequence.append(sequence[-1] + sequence[-2])
    return sequence

=== Documented Function ===
[... function with added documentation ...]

=== Test Cases ===
[... unittest test cases ...]
```

Final code has been saved to calculate_fibonacci_sequence_up.py

# Learning from This Design
This quasi-agent teaches us several important lessons about building LLM-powered systems:

1. **Prompt Chaining**: Breaking complex tasks into sequential steps makes them more manageable.

2. **Context Management**: Carefully controlling what the LLM sees helps maintain focus and consistency.

3. **Output Processing**: Having robust ways to extract and clean LLM output is crucial.

4. **Progressive Enhancement**: Building features iteratively (code → docs → tests) creates better results than trying to do everything at once.

These principles apply even when building more complex, fully agentic systems.

